In [ ]:
import torch
import time
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from networks.archive import DataAggregationArchive
from networks.ensemble import Ensemble
import numpy as np
from scipy import ndimage
import random
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

def CSVLineToVec(line):
    line_list = line.strip().replace("\n", "").split(",")
    float_list = []
    for i in line_list:
        float_list.append(float(i))
    float_list = np.array(float_list)
    return float_list

In [ ]:
# For single Sensor Baseline Model
# TRUTH_FILE = "validation-data-two-sensor.txt"
# TRUTH_FILE = "validation-data-baseline.txt"

"""
RANDOM - Only Random Weight Initialization
"""
# VALIDATION_FILE = "validation-data-baseline.txt"
# VALIDATION_DATA = SwarmDataset("../data/validation-easy-model", rank=0)
VALIDATION_FILE = "validation-data-two-sensor.txt"
VALIDATION_DATA = SwarmDataset("../data/validation-two-sensor-model", rank=0)
TESTING_FILE = "original-hand-labeled-classes.txt"
TESTING_DATA = SwarmDataset("../data/full-mini", rank=0)
ENSEMBLE_PATH = None

OUT = "../data/oracle"
validation_classes = []
with open(os.path.join(OUT, VALIDATION_FILE), "r") as f:
    lines = f.readlines()
    validation_classes = [-1 for i in range(len(lines))]
    for line in lines:
        triplet = CSVLineToVec(line)
        validation_classes[int(triplet[0])] = int(triplet[1])

testing_classes = []
with open(os.path.join(OUT, TESTING_FILE), "r") as f:
    lines = f.readlines()
    testing_classes = [-1 for i in range(len(lines))]
    for line in lines:
        triplet = CSVLineToVec(line)
        testing_classes[int(triplet[0])] = int(triplet[1])

validation_set = []
testing_set = []
for i, _class in enumerate(testing_classes):
    testing_set.append((i, _class))

for i, _class in enumerate(validation_classes):
    validation_set.append((i, _class))

print(len(validation_set), len(testing_set))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ensemble = Ensemble(size=3, output_size=5, lr=15e-4, learning_decay=0.7, decay_step=1, threshold=9.0, weight_decay=1e-4, new_model=True, init="Random")
if ENSEMBLE_PATH:
    ensemble.load_ensemble(ENSEMBLE_PATH, full=True)
ensemble.eval_mode()

MARGIN = 0

# for metric in ["TRAIN", "VAL"]:
for metric in ["VAL"]:
    a = []
    sampled_dataset = TESTING_DATA if metric == "TRAIN" else VALIDATION_DATA
    c_set = testing_set if metric == "TRAIN" else validation_set
    classes = testing_classes if metric == "TRAIN" else validation_classes

    # random.shuffle(c_set)
    print("=" * 20)
    print(f"{metric} results")
    print(f"Class Set of Size: {len(c_set)}")
    embedded_positions = []
    for j, c in enumerate(classes):
        _, embed = sampled_dataset[j][0], sampled_dataset[j][2]
        for k in range(len(embed)):
            embed[k] = abs(embed[k])
        embedded_positions.append(embed)

    # Evaluate Accuracy
    correct, total = 0, 0
    class_counts = {i:0 for i in range(max(classes) + 1)}
    class_accuracy = {i:0 for i in range(max(classes) + 1)}
    for l in range(len(c_set)):
        x, _classX = c_set[l]
        break_class_X = False
        if _classX == 0:
            continue
        for j in range(l, len(c_set)):
            y, _classY = c_set[j]
            if x == y or _classX != _classY:
                continue
            for k in range(len(c_set)):
                z, _classZ = c_set[k]
                # if _classZ == 0:
                #     continue
                if _classZ == _classX or x == z or y == z:
                    continue
                positive_dist = np.linalg.norm(embedded_positions[x] - embedded_positions[y])
                negative_dist = np.linalg.norm(embedded_positions[x] - embedded_positions[z])
                if positive_dist + MARGIN < negative_dist:
                    correct += 1
                    class_accuracy[_classX] += 1
                total += 1
                class_counts[_classX] += 1

    print(class_counts)
    print(f"CLASS ACCURACY (Out of {total} triplets):")
    for class_value in class_accuracy:
        if class_value == 0 or class_counts[class_value] == 0:
            continue
        print(f"{class_value}: {class_accuracy[class_value] * 100 / class_counts[class_value]}")
    acc = correct * 100 / total
    a.append(acc)
    print(f"Accuracy: {acc}")